In [ ]:
#!pip install langchain
#!pip install gradio==4.1.2

## initial bedrock client

In [1]:
import os
from typing import Optional
import boto3
import json
from botocore.config import Config
from botocore.config import Config
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.agents import Tool, AgentExecutor, AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.chains import LLMChain
from langchain.schema import BaseRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import BaseRetriever, Document
from typing import Any, Dict, List, Optional,Union
from langchain.utilities import SerpAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.memory import ConversationBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
import re


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
  
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]
        

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    client_kwargs["aws_access_key_id"] = os.environ.get("AWS_ACCESS_KEY_ID","")
    client_kwargs["aws_secret_access_key"] = os.environ.get("AWS_SECRET_ACCESS_KEY","")
    
    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client



## for aksk bedrock
def get_bedrock_aksk(secret_name='chatbot_bedrock', region_name = "us-west-2"):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['BEDROCK_ACCESS_KEY'],secret['BEDROCK_SECRET_KEY']

ACCESS_KEY, SECRET_KEY=get_bedrock_aksk()

#role based initial client#######
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "default"
#os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123"  # E.g. "arn:aws:..."
os.environ["AWS_ACCESS_KEY_ID"]=ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"]=SECRET_KEY


#新boto3 sdk只能session方式初始化bedrock
boto3_bedrock = get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

parameters_bedrock = {
    "max_tokens_to_sample": 2048,
    #"temperature": 0.5,
    "temperature": 0,
    #"top_k": 250,
    #"top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

bedrock_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs=parameters_bedrock)
###test the bedrock langchain integration###
#response=bedrock_llm.predict("我想去川西")
#response

Create new client
  Using region: us-west-2
  Using profile: default
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [17]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history", input_key='input', output_key="output")
cot_template = """
        You are an experienced AWS CLI expert who can effectively troubleshoot and resolve issues.
        Generate AWS CLI scripts to troubleshoot user's questions
        Ensure the script is clear, concise, and includes specific details about the expected user's question . Specify the length and format for any troubleshooting documentation that needs to be provided.
        ### Note: Please only respond if you have extensive experience with the AWS CLI and can confidently troubleshoot and resolve issues.
        user's question is in the following <question> tags:
        <question>
        {input}
        </question>
        """

awscli_template = """
        The following is a friendly conversation between a human and an AI. The AI will respond with plain string based on the AWS CLI script output 
        Context AWS CLI script ouput, use JSON formation or plan text format, output is:
        Here is my question:
          {input}
"""

prompt = PromptTemplate(
    input_variables=["input"], template=cot_template
)
bedrock_chain = LLMChain(llm=bedrock_llm, prompt=prompt, verbose=True)

* for test only

In [27]:
import re
gen_response=bedrock_chain.run("安全组放开了80端口的aws EC2实例有哪些?")
pattern = re.compile(r'`{3}bash\n(.*?)\n`{3}', re.DOTALL)
results = re.findall(pattern, gen_response)
print(type(results[0]))
print(results[0])



> Entering new LLMChain chain...
Prompt after formatting:

        You are an experienced AWS CLI expert who can effectively troubleshoot and resolve issues.
        Generate AWS CLI scripts to troubleshoot user's questions
        Ensure the script is clear, concise, and includes specific details about the expected user's question . Specify the length and format for any troubleshooting documentation that needs to be provided.
        ### Note: Please only respond if you have extensive experience with the AWS CLI and can confidently troubleshoot and resolve issues.
        user's question is in the following <question> tags:
        <question>
        安全组放开了80端口的aws EC2实例有哪些?
        </question>
        

> Finished chain.
<class 'str'>
# Get list of security groups
security_groups=$(aws ec2 describe-security-groups --query 'SecurityGroups[].GroupId' --output text)

# Loop through each security group
for sg in $security_groups; do

  # Get ingress rules for security group
  rules=$(a

### agent initial ####

In [29]:
import requests
import json
def execution_request(code:str):
    data = {
        "language": "awscli", 
        "version":"bash",
        "files":[{"name": "awscli-gen.py","content": code}],
        "stdin": "", 
        "args": [], 
        "compile_timeout": 10000, 
        "run_timeout": 3000, 
        "compile_memory_limit": -1, 
        "run_memory_limit": -1
    }

    response = requests.post("https://172.31.25.136:2000/api/v2/execute", json=data)
    result = response.json()
    return result


def awscli_code_gen(query:str):
    gen_response=bedrock_chain.run(query)
    pattern = re.compile(r'`{3}bash\n(.*?)\n`{3}', re.DOTALL)
    results = re.findall(pattern, gen_response)
    #print(type(results[0]))
    #print(results[0])
    return "```bash\n"+results[0]+"\n```"

runtime_tool = Tool(
    name="run the code with AWS runtime",
    func=execution_request,
    description="useful for when you need to run the code with AWS runtime to get final result",
)
codegen_tool = Tool(
    func=awscli_code_gen,
    name="AWS CLI script code generation",
    description="useful for when you need to generate aws cli scripte code",
)
custom_tool_list = [runtime_tool,codegen_tool]

In [30]:
customerized_instructions="""
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These are guidance on when to use a tool to solve a task, follow them strictly:
 - first use "AWS CLI script code generation" tool to generate aws script
 - then use "run code with AWS runtime" tool to execute the generated scripts and return result
"""

##use customerized outputparse to fix claude not match 
##langchain's openai ReAct template don't have 
##final answer issue 
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        #print("cur step's llm_output ==="+llm_output)
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output},
                log=llm_output,
            )
            #raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()
agent_executor = initialize_agent(custom_tool_list, bedrock_llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                                  verbose=True,max_iterations=3,
                                  handle_parsing_errors=True,
                                  memory=memory,
                                  agent_kwargs={
                                      "output_parser": output_parser,
                                      #'prefix':PREFIX,
                                      #'suffix':SUFFIX,
                                      'format_instructions':customerized_instructions
                                           })
query = "没有打tag的aws EC2实例有哪些？"
inputs={"input":query}
response = agent_executor.invoke(inputs)
#print(response)
response_output = response['output']
steps = response['intermediate_steps'] if response['intermediate_steps'] else None
stepMsgs = []
for step in steps:
    tool_name = None
    tool_input = None
    if action is not None:
        tool_name = step[0].tool
        tool_input = step[0].tool_input
        observation = step[1]
        stepMsgs.append((tool_name,tool_input,observation))
print("stepMsgs===\n"+stepMsgs)
    

#agent_executor.run("没有打tag的aws EC2实例有哪些？")






> Entering new AgentExecutor chain...
 Question: 没有打tag的aws EC2实例有哪些?

Thought: 要找到没有打tag的EC2实例,可以使用AWS CLI生成脚本,然后在AWS运行时环境中运行脚本获取结果。

Action: AWS CLI script code generation
Action Input: 生成一个脚本,列出所有没有打tag的EC2实例

> Entering new LLMChain chain...
Prompt after formatting:

        You are an experienced AWS CLI expert who can effectively troubleshoot and resolve issues.
        Generate AWS CLI scripts to troubleshoot user's questions
        Ensure the script is clear, concise, and includes specific details about the expected user's question . Specify the length and format for any troubleshooting documentation that needs to be provided.
        ### Note: Please only respond if you have extensive experience with the AWS CLI and can confidently troubleshoot and resolve issues.
        user's question is in the following <question> tags:
        <question>
        生成一个脚本,列出所有没有打tag的EC2实例
        </question>
        

> Finished chain.

Observation: ```bash
# List all EC2 instances
aws ec

SSLError: HTTPSConnectionPool(host='172.31.25.136', port=2000): Max retries exceeded with url: /api/v2/execute (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1007)')))